## <a id='toc1_1_'></a>[Solcast API](#toc0_)
Author : Saddik Imad
<br/>
Date : 23/03/2024

**Table of contents**<a id='toc0_'></a>    
- [Loading the API Key](#toc1_2_)    
- [Making multiple requests](#toc1_3_)    
- [To a dataframe](#toc1_4_)    

## <a id='toc1_2_'></a>[Loading the API Key](#toc0_)

Make sur to create a .env file and add `SOLCAST_API_KEY=` to it. Make sure to get the API Key from [here](https://toolkit.solcast.com.au/account/api-key).

In [1]:
import os
from dotenv import load_dotenv


# Load the API key from the .env file
load_dotenv()
key = os.getenv("SOLCAST_API_KEY")

## <a id='toc1_3_'></a>[Making multiple requests](#toc0_)

Solcast provides a wide range of services (forecast, live data, historic data, historic forecast, etc), make sure to visit the [docs](https://docs.solcast.com.au/) to learn more. Let's show how to get irradiance and weather Historical data by using the API, read about this part in this [section](https://docs.solcast.com.au/#9de907e7-a52f-4993-a0f0-5cffee78ad10) in the docs.

### Define the parameters for the request


In [9]:
latitude = -33.856784
longitude = 151.215297
duration = "P31D" # maximum duration is 31 days
format_ = "json"
time_zone = "utc"
output_parameters = "albedo,precipitation_rate,surface_pressure"

### Pass the API key in the Authorization header

In [4]:
headers = {
    "Authorization": f"Bearer {key}"
}

### Getting the list of dates

In [8]:
import pandas as pd


start_date = "2021-01-01"
end_date = "2022-01-01"

monthly_dates = pd.date_range(start=start_date, end=end_date, freq="MS")
monthly_dates = monthly_dates.strftime("%Y-%m-%d")
monthly_dates

Index(['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01',
       '2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01',
       '2021-11-01', '2021-12-01', '2022-01-01'],
      dtype='object')

### Putting everything together

In [10]:
import requests


responses = []

for date in monthly_dates:
    start = f"{date}T00:00:00.000Z"
    url = f"https://api.solcast.com.au/data/historic/radiation_and_weather?latitude={latitude}&longitude={longitude}&start={start}&duration={duration}&format={format_}&time_zone={time_zone}&output_parameters={output_parameters}"

    response = requests.get(url, headers=headers)
    responses.append(response)

## <a id='toc1_4_'></a>[To a dataframe](#toc0_)

Let's take the response and convert it to a dataframe.

In [13]:
import pandas as pd


data = pd.DataFrame()

for response in responses:
    data = pd.concat([data, pd.DataFrame(response.json()['estimated_actuals'])])

data = data.reset_index(drop=True)
data.head()

,albedo,precipitation_rate,surface_pressure,period_end,period
0,0.08,0.1,1029.4,2021-01-01T00:30:00.0000000Z,PT30M
1,0.08,0.1,1029.3,2021-01-01T01:00:00.0000000Z,PT30M
2,0.08,0.1,1029.1,2021-01-01T01:30:00.0000000Z,PT30M
3,0.08,0.1,1029.0,2021-01-01T02:00:00.0000000Z,PT30M
4,0.08,0.2,1028.9,2021-01-01T02:30:00.0000000Z,PT30M


In [14]:
data.shape

(19344, 5)